## 1. Importar módulos necesarios

In [1]:
# %% [markdown]
# ## 1. Importar módulos necesarios

import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.utils import to_categorical

2025-02-03 11:48:21.096841: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-03 11:48:21.099252: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-03 11:48:21.106773: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738579701.120029   21500 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738579701.123909   21500 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-03 11:48:21.136942: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

## 1.1 Comprobar si los paquetes se han cargado correctamente

In [3]:
# %% [markdown]
# ## 1.1 Comprobar si los paquetes se han cargado correctamente

from tensorflow.keras import Input

print("TensorFlow version:", tf.__version__)
print("Eager execution:", tf.executing_eagerly())

# Verificar si Keras funciona
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Input(shape=(20,)),  # Define explícitamente la capa de entrada
    Dense(10, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.summary()

TensorFlow version: 2.18.0
Eager execution: True


W0000 00:00:1738580062.986832   21500 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 221 (884.00 B)

 Trainable params: 221 (884.00 B)

 Non-trainable params: 0 (0.00 B)

## 2. Definir la estructura de los archivos de entrada

## 2. Definir la estructura de los archivos de entrada

**Archivo de Entrenamiento ("training_data.csv")**:
- Columnas 1 a 160: Genes de resistencia.
  - Valor: "" (vacío) si la cepa es wild type para ese gen.
  - Valor: Cadena de mutaciones separadas por comas, e.g., "A123T,V456G", si hay mutaciones.
- Columnas 161 a 163: "IMI_MIC", "AZT_MIC", "FEP_MIC" (valores numéricos, pero no se usan directamente en el modelo).
- Columnas 164 a 166: "IMI_eval", "AZT_eval", "FEP_eval" (evaluación clínica: "sensitive" o "resistant").
- Columna 167: "danger_profile" (por ejemplo, "sensitive", "MDR", "XDR").

**Archivo de Test ("test_data.csv")**:
- Columnas 1 a 160: Genes de resistencia (mismo formato).



In [ ]:
# %% [markdown]
# ## 3. Cargar y preprocesar los datos
# Leer el archivo de entrenamiento (1200 cepas)
train_df = pd.read_csv('training_data.csv')

# Suponemos que las primeras 160 columnas son genes de resistencia
gene_cols = train_df.columns[:68]

# --- Preprocesamiento de los genes: codificación multi-hot por gen ---
# La idea es, para cada gen, construir un vocabulario de mutaciones (observadas en el set de entrenamiento)
# y, para cada celda, crear un vector binario indicando la presencia de cada mutación.
def build_gene_vocab(df, gene_columns):
    vocab = {}
    for col in gene_columns:
        mutations = set()
        for val in df[col]:
            if pd.isna(val) or val.strip() == "":
                continue
            # Dividir las mutaciones por coma
            for mut in val.split(","):
                mut = mut.strip()
                if mut:
                    mutations.add(mut)
        # Ordenamos el vocabulario para tener un orden fijo
        vocab[col] = sorted(list(mutations))
    return vocab

gene_vocab = build_gene_vocab(train_df, gene_cols)

print('Archivo de lectura: train_df:')
print(train_df)

print('Columnas con genes de resistencia: gene_cols:')
print(gene_cols)

print('Vocabulario de mutaciones: gene_vocab')
print(gene_vocab)

In [4]:
# Función para codificar una fila: para cada gen, si la celda está vacía se retorna un vector de ceros,
# si tiene mutaciones, se asigna 1 en las posiciones correspondientes al vocabulario.
def encode_gene_row(row, gene_columns, vocab):
    features = []
    for col in gene_columns:
        gene_voc = vocab[col]
        vec = np.zeros(len(gene_voc), dtype=int)
        cell = row[col]
        if pd.isna(cell) or cell.strip() == "":
            # Wild type: vector de ceros
            pass
        else:
            mutations = [m.strip() for m in cell.split(",") if m.strip() != ""]
            for mut in mutations:
                if mut in gene_voc:
                    idx = gene_voc.index(mut)
                    vec[idx] = 1
        # Agregar el vector para este gen a la lista de características
        features.extend(vec.tolist())
    return features

# Aplicar la codificación a cada fila para los genes en el set de entrenamiento
X_train = train_df.apply(lambda row: encode_gene_row(row, gene_cols, gene_vocab), axis=1)
X_train = np.array(X_train.tolist())

# Preparar las etiquetas:
# Usaremos las evaluaciones clínicas de MIC para 3 antibióticos (IMI, AZT, FEP) y el perfil de peligrosidad.
# Suponemos que las columnas de evaluación están etiquetadas como "IMI_eval", "AZT_eval", "FEP_eval"
antibiotic_cols = ['IMI_eval', 'AZT_eval', 'FEP_eval']
y_antibiotics = train_df[antibiotic_cols]

# Para cada antibiótico, codificamos las etiquetas ("sensitive" o "resistant") en formato binario.
antibiotic_encoders = {}
y_antibiotics_encoded = {}
for col in antibiotic_cols:
    le = LabelEncoder()
    y_enc = le.fit_transform(y_antibiotics[col].astype(str))
    antibiotic_encoders[col] = le
    # Convertimos a formato one-hot (aunque en clasificación binaria se puede usar una sola neurona con sigmoide)
    y_antibiotics_encoded[col] = to_categorical(y_enc)

# Para el perfil de peligrosidad (por ejemplo: "sensitive", "MDR", "XDR")
le_profile = LabelEncoder()
y_profile = le_profile.fit_transform(train_df['danger_profile'].astype(str))
y_profile_cat = to_categorical(y_profile)

# Para este ejemplo, definiremos las salidas de la siguiente forma:
# - Para cada antibiótico: salida binaria (usaremos 1 neurona con sigmoide)  
#   Por ello, en lugar de one-hot, convertiremos las etiquetas a 0/1.
y_antibiotics_binary = {}
for col in antibiotic_cols:
    # Supongamos que "resistant" es 1 y "sensitive" es 0
    y_binary = (y_antibiotics[col].astype(str).str.lower() == "resistant").astype(int)
    y_antibiotics_binary[col] = y_binary.values.reshape(-1, 1)

# Consolidamos las salidas en un diccionario para el modelo multi-salida
y_train = {
    'IMI': y_antibiotics_binary['IMI_eval'],
    'AZT': y_antibiotics_binary['AZT_eval'],
    'FEP': y_antibiotics_binary['FEP_eval'],
    'profile': y_profile_cat
}

# Como nuestros datos de genes ya están en formato binario (multi-hot), la estandarización puede no ser necesaria.
X_train_input = X_train  # Forma: (1200, total_features) donde total_features = sum(len(vocab[gene]) for each gene)

# Cargar los datos de test (200 cepas), que solo tienen las 160 columnas de genes.
test_df = pd.read_csv('test_data.csv')
# Asegurarse de que los test tengan las mismas columnas y procesarlos de la misma forma
for col in gene_cols:
    test_df[col] = test_df[col].astype(str)
X_test = test_df.apply(lambda row: encode_gene_row(row, gene_cols, gene_vocab), axis=1)
X_test_input = np.array(X_test.tolist())



FileNotFoundError: [Errno 2] No such file or directory: 'training_data.csv'

In [ ]:
# %% [markdown]
# ## 4. Definir el modelo de redes neuronales multi-salida
#
# Utilizaremos el API funcional de Keras para construir un modelo que tenga dos tipos de salidas:
# - Tres salidas binarias para la evaluación de IMI, AZT y FEP.
# - Una salida multiclase para el perfil de peligrosidad (3 clases).
input_layer = Input(shape=(X_train_input.shape[1],), name='input')
x = Dense(256, activation='relu')(input_layer)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(64, activation='relu')(x)

# Salidas para antibióticos (una neurona cada uno con activación sigmoide)
out_IMI = Dense(1, activation='sigmoid', name='IMI')(x)
out_AZT = Dense(1, activation='sigmoid', name='AZT')(x)
out_FEP = Dense(1, activation='sigmoid', name='FEP')(x)

# Salida para perfil de peligrosidad (3 clases, activación softmax)
out_profile = Dense(3, activation='softmax', name='profile')(x)

# Definir el modelo con entradas y salidas
model = Model(inputs=input_layer, outputs=[out_IMI, out_AZT, out_FEP, out_profile])

# Compilar el modelo
model.compile(optimizer='adam',
              loss={'IMI': 'binary_crossentropy',
                    'AZT': 'binary_crossentropy',
                    'FEP': 'binary_crossentropy',
                    'profile': 'categorical_crossentropy'},
              metrics={'IMI': 'accuracy',
                       'AZT': 'accuracy',
                       'FEP': 'accuracy',
                       'profile': 'accuracy'})

model.summary()

In [ ]:
# %% [markdown]
# ## 5. Entrenar el modelo
history = model.fit(X_train_input, y_train,
                    epochs=50, batch_size=32, validation_split=0.2)



In [ ]:
# %% [markdown]
# ## 6. Evaluar el modelo en el set de test
#
# Para el set de test, se usa únicamente la información de las mutaciones para predecir:
# - La probabilidad de resistencia para cada antibiótico (se puede usar un umbral, p.ej. 0.5)
# - El perfil de peligrosidad (la clase con mayor probabilidad)
predictions = model.predict(X_test_input)

# Para las salidas binarias, aplicar umbral 0.5
pred_IMI = (predictions[0] > 0.5).astype(int)
pred_AZT = (predictions[1] > 0.5).astype(int)
pred_FEP = (predictions[2] > 0.5).astype(int)
# Para el perfil, se toma la clase de mayor probabilidad
pred_profile = np.argmax(predictions[3], axis=1)
pred_profile_labels = le_profile.inverse_transform(pred_profile)

# Mostrar algunas predicciones (ejemplo: perfil de peligrosidad)
print("Predicción del perfil de peligrosidad para las primeras 5 cepas del set de test:")
print(pred_profile_labels)